# A code to quickly check the *trajectory.nc* file content

Define here the name of the file to check

In [1]:
data_filename = '3_VarLight/2021-12-21/1/trajectory.nc'

In [2]:
from ipywidgets import interact, interactive
import utilities

import ipywidgets as widgets

from IPython.display import Markdown, display
from datetime import timedelta
import xarray as xr
import os, sys
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget


ModuleNotFoundError: No module named 'utilities'

In [3]:
root = '/Users/baptistelafoux/Google Drive/Pro/2020_PMMH_thèse_fish/5_PostProcessing/postProCode/python/free_swim_illum_var/'
root_server = '/Volumes/baptiste/data_labox/illuminance_variation/1_raw_data/'

original_video_filename = glob.glob(root_server + os.path.dirname(data_filename) + '/*noBG.mp4')[0]

ds = xr.open_dataset(root + 'cleaned/' + data_filename)
original_video_cap = cv2.VideoCapture(original_video_filename)

print('Video file :', original_video_filename)
print('Data file :', data_filename)



Video file : /Volumes/baptiste/data_labox/illuminance_variation/1_raw_data/3_VarLight/2021-12-21/1/2021-12-21_noBG.mp4
Data file : 3_VarLight/2021-12-21/1/trajectory.nc


Metadatas

In [4]:
dt = 1 / ds.fps

display(Markdown('**Video**'))
print('Frame rate : {} fps'.format(ds.fps))
print('Video length : {} images ('.format(ds.n_frames) + str(timedelta(seconds=int(ds.n_frames/ds.fps))) + ')')

display(Markdown('**Content**'))
print('Number of individuals : {} fish'.format(ds.n_fish))
print('Average body length : {:.2f} pxl'.format(ds.mean_BL_pxl))

**Video**

Frame rate : 5 fps
Video length : 21002 images (1:10:00)


**Content**

Number of individuals : 58 fish
Average body length : 43.16 pxl


In [5]:
trail_len = 5
delta_t = 100

fig, ax = plt.subplots(1, figsize=(16, 5))

ax = [ax]

def display_movie(t=trail_len):

    ax[0].clear()

    original_video_cap.set(cv2.CAP_PROP_POS_FRAMES, t)
    _, frame = original_video_cap.read()

    ax[0].imshow(frame)
    ax[0].invert_yaxis()
    ax[0].plot(ds.s[t-trail_len:t, :, 0] * ds.mean_BL_pxl,
              ds.s[t-trail_len:t, :, 1] * ds.mean_BL_pxl, linewidth=0.75)
    ax[0].plot(ds.s[t, :, 0] * ds.mean_BL_pxl, ds.s[t, :, 1] * ds.mean_BL_pxl, 'k.')

    #for focal in range(ds.n_fish):
    #   ax[0].annotate(str(focal), (ds.s[t, focal, 0], ds.s[t, focal, 1]), xytext=(2, 5), textcoords='offset points')

    #ax[0].axis('scaled')
    #ax[0].axis([0, ds.tank_size[0], 0, ds.tank_size[1]])

        
slider = interactive(display_movie, t=widgets.IntSlider(min=101, max=ds.n_frames-1, step=1))

display(slider)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=101, description='t', max=21001, min=101), Output()), _dom_classes=('wid…

In [56]:
r = ds.s - ds.center_of_mass
v = ds.v
N = ds.n_fish

R_max = 20
i = ds.ii_dist.mean(dim='neighbour') < R_max

print(np.sum(i) / 58)

print(i.shape) 
print(r.shape)
r *= i
v *= i

plt.figure()
plt.plot(ds.time, np.min(ds.ii_dist.mean(dim=['neighbour', 'fish']))/ds.ii_dist.mean(dim=['neighbour', 'fish']))

plt.figure()
plt.plot(ds.time, np.linalg.norm(r, axis=-1))
print(r.shape)

# we add an epilon to the norm of the velocity in case is it 0
rotation_parameter = np.sum(np.cross(r, v) / (np.linalg.norm(r, axis=2) * (np.linalg.norm(v, axis=2) + 10**(-9))), axis=1) / N

plt.figure()
plt.plot(ds.time, rotation_parameter)


<xarray.DataArray 'ii_dist' ()>
array(19714.)
(21002, 58)
(21002, 58, 2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(21002, 58, 2)


/var/folders/6g/_pvh03v53030m1dmqwzw8b0r0000gn/T/ipykernel_19694/1406645278.py:23: RuntimeWarning: invalid value encountered in true_divide
  rotation_parameter = np.sum(np.cross(r, v) / (np.linalg.norm(r, axis=2) * (np.linalg.norm(v, axis=2) + 10**(-9))), axis=1) / N


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
from src.utilities import dataloader
import matplotlib.pyplot as plt
import cv2 
import numpy as np 


%matplotlib widget
file = '/Volumes/baptiste/2019_PMMH_stage_AGimenez_collective/Collective/1_data_alicia/2_cured_datasets/serieB/60dpf/2/Tracking_Result_2/trajectory.nc'
file = '/Volumes/baptiste/2019_PMMH_stage_AGimenez_collective/Collective/1_data_alicia/2_cured_datasets/serieA/videos_adults_boite_190/6/Tracking_Result_6poissons/trajectory.nc'

ds = dataloader(file)


bg = plt.imread(ds.bg_filename)
circles = cv2.HoughCircles(bg, cv2.HOUGH_GRADIENT, 1, bg.shape[0] / 8,
                          param1=100, param2=30,
                          minRadius=100, maxRadius=1000)
plt.figure()
plt.imshow(bg, cmap='Greys_r')

circles = circles[0, 0, :]
c = plt.Circle(circles[0:2], circles[-1])
plt.gca().add_patch(c) 


ModuleNotFoundError: No module named 'src.utilities'

(1, 1)

In [45]:
circles[0:2].shape

(2,)

In [50]:
ds.s.to_numpy() - np.array([1, 3]) / ds.mean_BL_pxl


array([[[14.74387095, 19.41807711],
        [18.30854775, 18.90796209],
        [13.72091231, 18.10412236],
        [19.97430093, 16.9925502 ],
        [16.24517916, 19.95343525],
        [16.49075227, 19.33724081]],

       [[14.75583589, 19.42274636],
        [18.33933558, 18.89439209],
        [13.69763904, 18.11025075],
        [19.99443705, 16.9495056 ],
        [16.24517916, 19.95343525],
        [16.49075227, 19.33724081]],

       [[14.76940588, 19.42493507],
        [18.40324587, 18.86768985],
        [13.6749932 , 18.11521182],
        [20.04273456, 16.86151951],
        [16.24517916, 19.95343525],
        [16.49075227, 19.33724081]],

       ...,

       [[15.83734978, 16.12465427],
        [19.80212252, 14.59781112],
        [18.31453022, 11.36301649],
        [15.47825565, 11.13600363],
        [18.81165891, 11.48699954],
        [17.45290858, 10.62415223]],

       [[15.83516107, 16.14055889],
        [19.82006993, 14.61386165],
        [18.35261375, 11.33078411],
       

In [104]:
from src.utilities import dataloader_multiple
import glob 
paths = glob.glob('cleaned/*/**/2022*/*/', recursive=True) + \
    glob.glob('cleaned/*/**/2021-12-21*/*/', recursive=True)

ds = dataloader_multiple(paths)


################ Creating a large dataset to gather multiple experiments ################

///// WARNING : here we average all variables over 1 s \\\\ 

Merging file : cleaned/3_VarLight/cleaned/3_VarLight/2022-01-04/1//trajectory.nc - 1/6
Merging file : cleaned/3_VarLight/cleaned/3_VarLight/2022-01-04/2//trajectory.nc - 2/6
Merging file : cleaned/3_VarLight/cleaned/3_VarLight/2022-01-06/1//trajectory.nc - 3/6
Merging file : cleaned/3_VarLight/cleaned/3_VarLight/2022-01-06/2//trajectory.nc - 4/6
Merging file : cleaned/3_VarLight/cleaned/3_VarLight/2021-12-21/1//trajectory.nc - 5/6
Merging file : cleaned/3_VarLight/cleaned/3_VarLight/2021-12-21/2//trajectory.nc - 6/6

Merging all 6 datasets took 8.10 s


In [105]:
import matplotlib.pyplot as plt
from src.analysis import add_var_vs_light, add_var_vs_time, col
import numpy as np 
%matplotlib widget

fig, ax = plt.subplots() 
for exp in ds.experiment:
    DS = np.abs(ds.sel(experiment=exp))
    add_var_vs_light(np.abs(DS), 'rot_param', ax, correc=True, all_values=False, linestyle='-', marker=None)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/baptistelafoux/Google Drive/Pro/2020_PMMH_thèse_fish/5_PostProcessing/postProCode/python/free_swim_illum_var/src/analysis.py:39: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  p = ax.plot(ds.light, ds[var], **kwargs)


AttributeError: 'Line2D' object has no property 'error_plot'

In [50]:
from src.analysis import add_var_vs_light, add_var_vs_time, col
from src.utilities import dataloader, xarray_vector_norm

ds = dataloader('cleaned/3_VarLight/2022-01-06/1/trajectory.nc')


In [98]:
r = ds.s - ds.center_of_mass
v = ds.v 

r_min = xarray_vector_norm(r, dim='space').min(dim='fish')
r_max = xarray_vector_norm(r, dim='space').max(dim='fish')

C = 1 - (xarray_vector_norm(r, dim='space') - r_min) / (r_max - r_min)


In [99]:
norm_r = np.linalg.norm(r, axis=2)
norm_v = np.linalg.norm(v, axis=2) + 10**(-9)

rp = np.mean(np.cross(r, v) / (norm_r *  norm_v) * C, axis=1)


In [102]:
fig, ax = plt.subplots() 

#ax.plot(np.abs(rp)[::100])
#ax.plot(np.abs(ds.rot_param)[::100])
ax.plot(C.mean(dim='fish')[::100], 'k')
""" ax.plot(ds.time, norm_r.mean(axis=-1))
ax.plot(ds.time, r_min)
ax.plot(ds.time, r_max) """
#ax.plot(ds.time, ds.light)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

' ax.plot(ds.time, norm_r.mean(axis=-1))\nax.plot(ds.time, r_min)\nax.plot(ds.time, r_max) '

In [77]:
fig, ax = plt.subplots() 

ax.plot(ds.s[..., 0], ds.s[..., 1], 'r.', alpha=0.01, markersize=1)
ax.plot(ds.center_of_mass[..., 0], ds.center_of_mass[..., 1], 'k-', linewidth=0.6, alpha=0.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …